<a href="https://colab.research.google.com/github/antonysama/exploratory/blob/master/GasRFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install pkgs
# install.packages(c('readxl', 'lubridate', 'psych', dependencies = TRUE, repos = "https://cran.rstudio.com/"))
x<-c('readxl', 'lubridate', 'psych', 'dplyr')
lapply(x, require, character.only = TRUE)

Loading required package: readxl

Loading required package: lubridate


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Loading required package: psych

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

In [2]:
df<-read_excel('Gas_RFM.xlsx', sheet = "Calculation Monthly",skip =6  )
head(df,1)

Date,Fiscal_Year,Fiscal_Quarter,Cal_Year,EAD_Match,EAD_Match_Now,EAD_Annual_Match,RFM_Match,RFMAlternate_Match,RFM_Annual_Match,⋯,AllowCostRestrictAdjusted_CAD,AllowCostRestrictAdjusted_000CAD,Actl_TotalGCA_CAD,Fcst_TotalGCA_CAD,TotalGCA_CAD,TotalGCA_000CAD,Actl_NRRGasNGLs_000CAD,Fcst_NRRGasNGLs_000CAD,NRRGasNGLs_000CAD,EffRoyaltyRate_%
<dttm>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2018-01-01,2017 / 2018,Qtr4,2018,109,109,10,NA,13,NA,⋯,8118163,8118.163,203121203,NA,203121203,203121.2,7358.149,NA,7358.149,0.00625018


In [ ]:
#Inpect data quality & types  ---- this was TOO BIG to inspect, but the pkg runs faster than skimr
# describe(df) # pckg psych

In [3]:
#Subset fiscal years and aggregate the mean for C5 spec price
df_a <- subset(df, df$'Fiscal_Year' == "2022 / 2023" | df$'Fiscal_Year' == "2023 / 2024")

aggregate(df_a$'C5RefPriceSpec_CAD/M3', by=list(df_a$'Fiscal_Year'),mean)

Group.1,x
<chr>,<dbl>
2022 / 2023,719.8525
2023 / 2024,594.7277


In [4]:
#inspect column type(Optionsl)
str(df_a$'C5RefPriceSpec_CAD/M3')

 num [1:24] 817 860 865 731 692 ...


In [53]:
#Subset fiscal years and aggregate the sum for C5 production and mean for C5 spec price
df_a %>%
  group_by(Fiscal_Year) %>%
  summarize(across(ProdC5TotalAll_000M3, sum), across('C5RefPriceSpec_CAD/M3', mean))



Fiscal_Year,ProdC5TotalAll_000M3,C5RefPriceSpec_CAD/M3
<chr>,<dbl>,<dbl>
2022 / 2023,15228.73,719.8525
2023 / 2024,14366.04,594.7277


In [5]:
#Also, create a new fscl_yr column, a numeric one as opposed to above chr (Optional)
df$fscl_yr <- ifelse ( month(df$Date)<4, year(df$Date)-1, year(df$Date) )

#Inspect both fscl year columns
head(df[, c("fscl_yr", "Fiscal_Year")])

fscl_yr,Fiscal_Year
<dbl>,<chr>
2017,2017 / 2018
2017,2017 / 2018
2017,2017 / 2018
2018,2018 / 2019
2018,2018 / 2019
2018,2018 / 2019
